In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

class ReducedRankRegression(nn.Module):
    def __init__(self, p, q, r):
        """
        :param p: Number of predictors
        :param q: Number of responses
        :param r: Reduced rank
        """
        super(ReducedRankRegression, self).__init__()
        # self.W = nn.Parameter(torch.randn(p, r))
        # self.C = nn.Parameter(torch.randn(r, q))

        self.W = torch.ones((p,r))
        self.W.requires_grad_(True)  
        # nn.Parameter(torch.randn(p, r))
        # self.C = nn.Parameter(torch.randn(r, q))
        self.C = torch.ones((r,q))
        self.C.requires_grad_(True)
    
    def forward(self, X):
        return X @ self.W @ self.C

In [2]:
# Load the data
import pickle
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as netcdf

with open('ssp585_time_series.pkl', 'rb') as f:
    dic_ssp585 = pickle.load(f)

In [3]:
import os 

# Get the list of all files and directories
path = "/net/atmos/data/cmip6-ng/tos/ann/g025"
dir_list = os.listdir(path)

print("Files and directories in '", path, "' :")

list_model = []
list_forcing = []

for idx, file in enumerate(dir_list):

    file_split = file.split("_")
    
    # extract model names
    model_name = file_split[2]
    forcing = file_split[3]
    run_name = file_split[4]
    
    list_model.append(model_name)
    list_forcing.append(forcing)
    
model_names = list(set(list_model))
forcing_names = list(set(list_forcing))

Files and directories in ' /net/atmos/data/cmip6-ng/tos/ann/g025 ' :


In [4]:
import netCDF4 as netcdf

# define the file
file = '/net/h2o/climphys3/simondi/cope-analysis/data/erss/sst_annual_g050_mean_19812014_centered.nc'

# read the dataset
file2read = netcdf.Dataset(file,'r')

# load longitude, latitude and sst monthly means
lon = np.array(file2read.variables['lon'][:])
lat = np.array(file2read.variables['lat'][:])
sst = np.array(file2read.variables['sst'])

# define grid
lat_grid, lon_grid = np.meshgrid(lat, lon, indexing='ij')

time_period = 33
grid_lat_size = lat.shape[0]
grid_lon_size = lon.shape[0]

In [5]:
import skimage

# first filter out the models that do not contain ensemble members 
dic_reduced_ssp585 = {}

for m in list(dic_ssp585.keys()):
    if len(dic_ssp585[m].keys()) > 2:
        dic_reduced_ssp585[m] = dic_ssp585[m].copy()
        for idx_i, i in enumerate(dic_ssp585[m].keys()):
            dic_reduced_ssp585[m][i] = skimage.transform.downscale_local_mean(dic_reduced_ssp585[m][i],(1,2,2))
            lat_size = dic_reduced_ssp585[m][i][0,:,:].shape[0]
            lon_size = dic_reduced_ssp585[m][i][0,:,:].shape[1]

In [6]:
nan_idx = []
for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    for idx_i,i in enumerate(dic_reduced_ssp585[m].keys()):    
        nan_idx_tmp = list(np.where(np.isnan(dic_reduced_ssp585[m][i][0,:,:].ravel())==True)[0])
        nan_idx = list(set(nan_idx) | set(nan_idx_tmp))

notnan_idx = list(set(list(range(lon_size*lat_size))) - set(nan_idx))

In [7]:
# second, for each model we compute the anomalies 
dic_processed_ssp585 = {}

import numpy as np

for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    dic_processed_ssp585[m] = dic_reduced_ssp585[m].copy()
    
    mean_ref_ensemble = 0
    y_tmp = np.zeros((len(dic_reduced_ssp585[m].keys()),time_period, lat_size*lon_size))
    
    for idx_i, i in enumerate(dic_reduced_ssp585[m].keys()):
        y_tmp[idx_i,:,:] = dic_reduced_ssp585[m][i][131:164,:,:].copy().reshape(time_period, lat_size*lon_size)
        y_tmp[idx_i,:,nan_idx] = float('nan')
           
        if idx_i == 0:
            mean_ref_ensemble = np.nanmean(y_tmp[idx_i,:,:],axis=0)/ len(dic_reduced_ssp585[m].keys())
        else:
            mean_ref_ensemble += np.nanmean(y_tmp[idx_i,:,:],axis=0)/ len(dic_reduced_ssp585[m].keys())

    for idx_i, i in enumerate(dic_processed_ssp585[m].keys()):
        dic_processed_ssp585[m][i] = y_tmp[idx_i,:,:] - mean_ref_ensemble

/tmp/ipykernel_15042/2788081108.py:17: RuntimeWarning: Mean of empty slice
  mean_ref_ensemble = np.nanmean(y_tmp[idx_i,:,:],axis=0)/ len(dic_reduced_ssp585[m].keys())
/tmp/ipykernel_15042/2788081108.py:19: RuntimeWarning: Mean of empty slice
  mean_ref_ensemble += np.nanmean(y_tmp[idx_i,:,:],axis=0)/ len(dic_reduced_ssp585[m].keys())


In [8]:
# compute the forced response
dic_forced_response_ssp585 = dict({})
dic_forced_response_ssp585_tmp = {}

for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    dic_forced_response_ssp585[m] = dic_reduced_ssp585[m].copy()
    dic_forced_response_ssp585_tmp[m] = dic_reduced_ssp585[m].copy()

    for idx_i, i in enumerate(dic_forced_response_ssp585[m].keys()):
        
        y_tmp = dic_reduced_ssp585[m][i][131:164,:,:].copy().reshape(time_period, lat_size*lon_size)
        y_tmp[:,nan_idx] = float('nan')

        if idx_i == 0:
            mean_spatial_ensemble = y_tmp/ len(dic_forced_response_ssp585[m].keys())
        else:
            mean_spatial_ensemble += y_tmp/ len(dic_forced_response_ssp585[m].keys())

    for idx_i, i in enumerate(dic_forced_response_ssp585[m].keys()):        
        dic_forced_response_ssp585[m][i] = mean_spatial_ensemble - np.nanmean(mean_spatial_ensemble)
    # dic_forced_response_ssp585_tmp[m] = mean_spatial_ensemble - np.nanmean(mean_spatial_ensemble)

In [9]:
y_forced_response = {}
x_predictor = {}

for idx_m,m in enumerate(dic_processed_ssp585.keys()):
    y_forced_response[m] = {}
    x_predictor[m] = {}

    y_forced_response[m] = dic_forced_response_ssp585_tmp[m]
    
    for idx_i, i in enumerate(dic_forced_response_ssp585[m].keys()):
        
        x_predictor[m][i] = dic_processed_ssp585[m][i]
        x_predictor[m][i][:,nan_idx] = float('nan')

In [10]:
# # compute the variance
# variance_processed_ssp585 = {}
# std_processed_ssp585 = {}
# for idx_m,m in enumerate(x_predictor.keys()):
#     variance_processed_ssp585[m] = {}
#     arr_tmp = np.zeros((len(x_predictor[m].keys()),33))
    
#     for idx_i, i in enumerate(list(x_predictor[m].keys())):
#         arr_tmp[idx_i,:] = np.nanmean(x_predictor[m][i],axis=1)

#     arr_tmp_values = np.zeros((len(x_predictor[m].keys()),33))
#     for idx_i, i in enumerate(x_predictor[m].keys()):
#         arr_tmp_values[idx_i,:] = (y_forced_response[m][i] - arr_tmp[idx_i,:])**2

#     variance_processed_ssp585[m] = torch.mean(torch.nanmean(torch.from_numpy(arr_tmp_values),axis=0))

In [11]:
y_forced_response_concatenate = {}
x_predictor_concatenate = {}
count_x = 0


for idx_m,m in enumerate(dic_processed_ssp585.keys()):
    y_forced_response_concatenate[m] = 0
    x_predictor_concatenate[m] = 0

    for idx_i, i in enumerate(dic_forced_response_ssp585[m].keys()):
        count_x += len(dic_processed_ssp585[m].keys())*33
        
        if idx_i ==0:
            y_forced_response_concatenate[m] = dic_forced_response_ssp585[m][i]
            x_predictor_concatenate[m] = dic_processed_ssp585[m][i]
        else:
            y_forced_response_concatenate[m] = np.concatenate([y_forced_response_concatenate[m],dic_forced_response_ssp585[m][i]])
            x_predictor_concatenate[m] = np.concatenate([x_predictor_concatenate[m], dic_processed_ssp585[m][i]],axis=0)  
    x_predictor_concatenate[m][:,nan_idx] = float('nan')
    y_forced_response_concatenate[m][:,nan_idx] = float('nan')

In [12]:
# Data preprocessing
x_train = {}
y_train = {}

for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    x_train[m] = torch.nan_to_num(torch.from_numpy(x_predictor_concatenate[m])).to(torch.float64)
    y_train[m] = torch.nan_to_num(torch.from_numpy(y_forced_response_concatenate[m])).to(torch.float64)

In [13]:
# We want X in R^{grid x runs*time steps}
x_tmp = torch.zeros(count_x, lat_size*lon_size)
y_tmp = torch.zeros(count_x, lat_size*lon_size)

m0 = 'KACE-1-0-G'
x_test = torch.zeros(len(x_predictor[m0].keys())*33, lat_size*lon_size)
y_test = torch.zeros(len(x_predictor[m0].keys())*33, lat_size*lon_size)

count_tmp =0

for idx_m,m in enumerate(dic_reduced_ssp585.keys()):
    if m != 'KACE-1-0-G':

        if count_tmp ==0:
            x_tmp[:x_train[m].shape[0],:] = x_train[m]
            y_tmp[:y_train[m].shape[0],:] = y_train[m]
            count_tmp = x_train[m].shape[0]
    
        else:
            x_tmp[count_tmp:count_tmp+x_train[m].shape[0],:] = x_train[m]
            y_tmp[count_tmp:count_tmp+y_train[m].shape[0],:] = y_train[m]
            count_tmp = x_train[m].shape[0]

    else: 
        x_test = x_train[m]
        y_test = y_train[m]

In [ ]:
n_samples = count_x
n_predictors = lat_size*lon_size
n_responses = lat_size*lon_size
rank = 5

lambda_ = 1.0

# Define the model
model = ReducedRankRegression(n_predictors, n_responses, rank)
    
# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam([model.W,model.C], lr=0.1)
    
# Training loop
n_epochs = 1000
for epoch in range(n_epochs):
    model.train()
        
    # Forward pass
    y_pred = model(x_tmp)
        
    # Compute loss
    # loss = criterion(y_pred,y_tmp) + lambda_*torch.norm(torch.matmul(model.W,model.C),p='fro')
    loss = torch.norm(y_pred-y_tmp, p='fro')**2 + lambda_*torch.norm(torch.matmul(model.W,model.C),p='fro')
    # + lambda_*torch.norm(model.W, p='2')**2 
    
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    if (epoch+1) % 2 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')
        # print(torch.mean((y_pred - y_tmp)**2))
    
print("Training complete.")

Epoch [2/1000], Loss: 6314593353728.0000
Epoch [4/1000], Loss: 2411794530304.0000
Epoch [6/1000], Loss: 782218625024.0000
Epoch [8/1000], Loss: 212701200384.0000
Epoch [10/1000], Loss: 46059700224.0000
Epoch [12/1000], Loss: 7020828672.0000
Epoch [14/1000], Loss: 770532096.0000
Epoch [16/1000], Loss: 370750240.0000
Epoch [18/1000], Loss: 395428704.0000
Epoch [20/1000], Loss: 888042688.0000
Epoch [22/1000], Loss: 2572166144.0000
Epoch [24/1000], Loss: 5730219520.0000
Epoch [26/1000], Loss: 10138062848.0000
Epoch [28/1000], Loss: 15249319936.0000
Epoch [30/1000], Loss: 20371871744.0000
Epoch [32/1000], Loss: 24885102592.0000
Epoch [34/1000], Loss: 28297017344.0000
Epoch [36/1000], Loss: 30344431616.0000
Epoch [38/1000], Loss: 30978705408.0000
Epoch [40/1000], Loss: 30334296064.0000
Epoch [42/1000], Loss: 28678199296.0000
Epoch [44/1000], Loss: 26316711936.0000
Epoch [46/1000], Loss: 23566059520.0000
Epoch [48/1000], Loss: 20688119808.0000
Epoch [50/1000], Loss: 17875472384.0000
Epoch [52

In [ ]:
M  = torch.matmul(model.W.detach(), model.C.detach())
MX = model.C[1,:].detach()

In [ ]:
# define beta to plot
M_tmp = MX.detach().clone()
M_tmp[nan_idx] = float('nan')
M_tmp = M_tmp.detach().numpy().reshape(lat_size,lon_size)

# define robust beta
# beta_robust_tmp = beta_robust.detach().clone()
# beta_robust_tmp[nan_idx] = float('nan')
# beta_robust_tmp = beta_robust_tmp.detach().numpy().reshape(lat.shape[0],lon.shape[0])

fig0 = plt.figure(figsize=(16,16))           

ax0 = fig0.add_subplot(2, 2, 1)        
ax0.set_title(r'Reduced rank regression $WC$', size=7,pad=3.0)
im0 = ax0.pcolormesh(lon_grid,lat_grid,M_tmp)
plt.colorbar(im0, ax=ax0, shrink=0.3)
ax0.set_xlabel(r'x', size=7)
ax0.set_ylabel(r'y', size=7)

# ax1 = fig0.add_subplot(2, 2, 2)        
# ax1.set_title(r'Robust regression coefficient $\beta_{\mathrm{rob}}$', size=7,pad=3.0)
# im1 = ax1.pcolormesh(lon_grid,lat_grid,beta_robust_tmp,vmin=-0.00,vmax = 0.001)
# plt.colorbar(im1, ax=ax1, shrink=0.3)
# ax1.set_xlabel(r'x', size=7)
# ax1.set_ylabel(r'y', size=7)

plt.show()

# Use the closed-form reduced rank regression

In [ ]:
def ols_estimator(X, Y):
    """
    Perform Reduced Rank Regression using the closed-form solution.
    
    :param X: Predictor matrix of shape (n_samples, n_predictors)
    :param Y: Response matrix of shape (n_samples, n_responses)
    :return: Coefficient matrix B of shape (n_predictors, n_responses)
    """

    # Compute the covariance matrices
    SXX = X.T @ X
    SXY = X.T @ Y
    SYY = Y.T @ Y

    B = torch.linalg.solve(SXX + 0.001*torch.eye(SXX.shape[0],SXX.shape[1]), SXY)
    
    return B

def rrr_estimator(X, Y,rank):
    """
    Perform Reduced Rank Regression using the closed-form solution.
    
    :param X: Predictor matrix of shape (n_samples, n_predictors)
    :param Y: Response matrix of shape (n_samples, n_responses)
    :param rank: Desired rank for the regression
    :return: Coefficient matrix B of shape (n_predictors, n_responses)
    """
    
    # Compute the covariance matrices
    SXX = X.T @ X
    SXY = X.T @ Y
    SYY = Y.T @ Y

    # compute ols estimator
    B = ols_estimator(X, Y)

    # Compute the SVD of the cross-covariance matrix
    U, D, Vt = torch.svd(X @ B)
    
    # # Select the top-rank components
    U_r = U[:, :rank]
    Vt_r = Vt[:rank, :]
    
    # Compute the low-rank coefficient matrix
    B_tmp = B @ Vt_r.T @ Vt_r
    
    return B

In [ ]:
B = rrr_estimator(x_tmp, y_tmp,50).to(torch.float64)

In [ ]:
# # Compute the SVD of the cross-covariance matrix
# U, D, Vt = torch.svd(B)

In [ ]:
y_pred = x_test @ B
print("Norm", torch.norm(y_test-y_pred,p='fro'))

In [ ]:
# define beta to plot
M_tmp = Ypred_1.detach().clone()
M_tmp[nan_idx] = float('nan')
M_tmp = M_tmp.detach().numpy().reshape(lat_size,lon_size)

# define robust beta
# beta_robust_tmp = beta_robust.detach().clone()
# beta_robust_tmp[nan_idx] = float('nan')
# beta_robust_tmp = beta_robust_tmp.detach().numpy().reshape(lat.shape[0],lon.shape[0])

fig0 = plt.figure(figsize=(16,16))           

ax0 = fig0.add_subplot(2, 2, 1)        
ax0.set_title(r'Reduced rank regression $WC$', size=7,pad=3.0)
im0 = ax0.pcolormesh(lon_grid,lat_grid,M_tmp,vmin=-0.00,vmax = 0.001)
plt.colorbar(im0, ax=ax0, shrink=0.3)
ax0.set_xlabel(r'x', size=7)
ax0.set_ylabel(r'y', size=7)

# ax1 = fig0.add_subplot(2, 2, 2)        
# ax1.set_title(r'Robust regression coefficient $\beta_{\mathrm{rob}}$', size=7,pad=3.0)
# im1 = ax1.pcolormesh(lon_grid,lat_grid,beta_robust_tmp,vmin=-0.00,vmax = 0.001)
# plt.colorbar(im1, ax=ax1, shrink=0.3)
# ax1.set_xlabel(r'x', size=7)
# ax1.set_ylabel(r'y', size=7)

plt.show()

In [ ]:
def reduced_rank_regression(X, Y, rank):
    """
    Perform Reduced Rank Regression using the closed-form solution.
    
    :param X: Predictor matrix of shape (n_samples, n_predictors)
    :param Y: Response matrix of shape (n_samples, n_responses)
    :param rank: Desired rank for the regression
    :return: Coefficient matrix B of shape (n_predictors, n_responses)
    """
    # Center the data
    X_mean = torch.mean(X, dim=0)
    Y_mean = torch.mean(Y, dim=0)
    X_centered = X
    Y_centered = Y
    
    # Compute the covariance matrices
    SXX = X_centered.T @ X_centered
    SXY = X_centered.T @ Y_centered
    SYY = Y_centered.T @ Y_centered
    
    # Compute the SVD of the cross-covariance matrix
    U, D, Vt = torch.svd(SXY)
    
    # Select the top-rank components
    U_r = U[:, :rank]
    Vt_r = Vt[:rank, :]
    
    # Compute the low-rank coefficient matrix
    B = U_r @ torch.diag(D[:rank]) @ Vt_r
    
    return B

In [ ]:
rank = 20
B= reduced_rank_regression(x_tmp, y_tmp, rank)
B_tmp= B[1,:].detach()

In [ ]:
# define beta to plot
M_tmp = B_tmp.detach().clone()
M_tmp[nan_idx] = float('nan')
M_tmp = M_tmp.detach().numpy().reshape(lat_size,lon_size)

# define robust beta
# beta_robust_tmp = beta_robust.detach().clone()
# beta_robust_tmp[nan_idx] = float('nan')
# beta_robust_tmp = beta_robust_tmp.detach().numpy().reshape(lat.shape[0],lon.shape[0])

fig0 = plt.figure(figsize=(16,16))           

ax0 = fig0.add_subplot(2, 2, 1)        
ax0.set_title(r'Reduced rank regression $WC$', size=7,pad=3.0)
im0 = ax0.pcolormesh(lon_grid,lat_grid,M_tmp,vmin=-0.00,vmax = 0.001)
plt.colorbar(im0, ax=ax0, shrink=0.3)
ax0.set_xlabel(r'x', size=7)
ax0.set_ylabel(r'y', size=7)

# ax1 = fig0.add_subplot(2, 2, 2)        
# ax1.set_title(r'Robust regression coefficient $\beta_{\mathrm{rob}}$', size=7,pad=3.0)
# im1 = ax1.pcolormesh(lon_grid,lat_grid,beta_robust_tmp,vmin=-0.00,vmax = 0.001)
# plt.colorbar(im1, ax=ax1, shrink=0.3)
# ax1.set_xlabel(r'x', size=7)
# ax1.set_ylabel(r'y', size=7)

plt.show()